# Atividade 1: Otimização Pythonic e Benchmarking de Performance

## Objetivo
Este notebook demonstra a importância da otimização de código em Python para Ciência de Dados.
Faremos um **benchmarking** de performance, comparando a execução de tarefas comuns (cálculo de faturamento) usando:

1.  **Loop FOR Tradicional:** Abordagem procedural.
2.  **List Comprehension:** Abordagem Pythonic e otimizada.
3.  **Funções Lambda:** Uso eficiente em conjunto com métodos de iteração (como `map`).

O teste será realizado em um volume de **500.000 linhas** de dados simulados.

In [1]:
import pandas as pd
import timeit
import numpy as np

df = pd.read_csv('../data/df_produtos_500k.csv') 

print(f"Dados carregados: {len(df)} linhas.")
print("\nPrimeiras 5 linhas do dataset:")
display(df.head())

Dados carregados: 500000 linhas.

Primeiras 5 linhas do dataset:


,produto,preco_unitario,quantidade_vendida,categoria
0,Produto_297,311.43,1467,Livros
1,Produto_509,413.07,644,Livros
2,Produto_587,46.87,1740,Alimentos
3,Produto_546,45.28,135,Roupas
4,Produto_693,294.53,1349,Eletronicos


In [2]:
# --- MÉTODO 1: LOOP FOR TRADICIONAL ---
def calcular_faturamento_loop(df):
    faturamento = []
    # Utiliza o zip para iterar sobre as colunas com performance aceitável
    for preco, qtd in zip(df['preco_unitario'], df['quantidade_vendida']):
        faturamento.append(preco * qtd)
    return faturamento

# Medição do tempo
tempo_loop = timeit.timeit(
    lambda: calcular_faturamento_loop(df),
    number=10 # Repetimos 10 vezes para uma média mais estável
)
tempo_medio_loop = tempo_loop / 10
print(f"\nTempo médio (Loop FOR): {tempo_medio_loop:.5f} segundos")


Tempo médio (Loop FOR): 0.09367 segundos


In [ ]:
# --- MÉTODO 2: LIST COMPREHENSION ---
def calcular_faturamento_comprehension(df):
    # A List Comprehension deve substituir o loop FOR acima em uma única linha
    faturamento = [preco * qtd for preco, qtd in zip(df['preco_unitario'], df['quantidade_vendida'])]
    return faturamento

# Medição do tempo
tempo_comprehension = timeit.timeit(
    lambda: calcular_faturamento_comprehension(df),
    number=10
)
tempo_medio_comprehension = tempo_comprehension / 10
print(f"Tempo médio (List Comprehension): {tempo_medio_comprehension:.5f} segundos")

Tempo médio (List Comprehension): 0.09036 segundos


In [6]:
# --- MÉTODO 3: Otimização com map() e Lambda ---
def calcular_faturamento_map_lambda(df):
    # O map() aplica a função lambda a cada par de elementos (preco, qtd)
    # O list() é necessário para converter o objeto map (que é lazy) em uma lista para comparação
    faturamento = list(map(
        lambda x: x[0] * x[1], 
        zip(df['preco_unitario'], df['quantidade_vendida'])
    ))
    return faturamento

# Medição do tempo
tempo_map_lambda = timeit.timeit(
    lambda:calcular_faturamento_map_lambda(df),
    number = 10
)

tempo_medio_map_lambda = tempo_map_lambda / 10
print(f"Tempo médio (map_lambda): {tempo_medio_map_lambda:.5f} segundos")

Tempo médio (map_lambda): 0.10778 segundos


In [8]:
diferenca_percentual_comprehension = ((tempo_medio_loop - tempo_medio_comprehension) / tempo_medio_loop) * 100
diferenca_vs_loop_map = ((tempo_medio_loop - tempo_medio_map_lambda) / tempo_medio_loop) * 100
diferenca_comprehension_vs_map = ((tempo_medio_map_lambda - tempo_medio_comprehension) / tempo_medio_map_lambda) * 100


print(f"\nDiferenca de performance do List Comprehension x Loop For: {diferenca_percentual_comprehension:.2f}%.")
print(f"Diferenca de performance do Map+Lambda x Loop For: {diferenca_vs_loop_map:.2f}%.")
print(f"Diferenca de performance do List Comprehension x Map+Lambda: {diferenca_comprehension_vs_map:.2f}%.")



Diferenca de performance do List Comprehension x Loop For: 3.53%.
Diferenca de performance do Map+Lambda x Loop For: -15.07%.
Diferenca de performance do List Comprehension x Map+Lambda: 16.17%.


A List Comprehension apresentou uma performance superior ao Loop For, ainda que modesta. Já no caso do Map/Lambda, a performance foi inferior tanto em relação ao Loop FOR, quanto em relação à List Comprehension. Apesar do método Map/Lambda ser um "pythonic", para este exemplo de operação simples, as repetidas chamadas da função lambda e a criação do objeto _lazy_ do `map` acabaram sobrecarregando o método.
